In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
import numpy as np

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

enlighten the front panels uses a lot of energy and cause glitches on the power rail. never the less an eluminated panel looks much better and the different types of connectors can be marked with different colors.


|color|type|[V]|
|---  |--- |---|
|<button class="button is-danger">red</button>| | audio signal | +-5 |
|<button class="button is-success">green</button>| | cv signal | +-5 |
|<button class="button is-warning">yellow</button>| | cv pulse or gate | +10 |
|<button class="button is-text">white</button>| | cv pulse or gate | +10 |


**the chip should be powered by +5V/GND. With higher voltage the heat dissipation will be to big and the chip will be damaged.**

# dc led driver

when there is a DC signal which is positive only and the signal has to be buffered the simgle transistor driver comes in handy. It is built around the common base transistor amplifier. The Led can be connected to the collector of the transisitor. The current for the LED can be set with Rx and calculated using simple ohms law.

In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=0.9)

d.add(elm.Dot('left', lftlabel='IN', open=True))
d.add(elm.Resistor(label='$R_1$\n100k'))
Q = d.add(elm.BjtNpn(circle='True', label='$Q_1$', to='base'))

d.add(elm.LED("down", label='$D_1$', at=Q.emitter))
d.add(elm.Ground)

d.add(elm.Resistor("up", label="R$_2$", at=Q.collector))
d.add(elm.Vdd(label="+15V"))

d.draw()

This is the first setup with the 4069 as voltage follower. C1 and C3 are the dc blocking capacitors. When we choose R1 and R2 as 100kOhm we would excpect a gain of one. 

In [ ]:
V = 5 - 0.2 - 1.7
Ie = V / 5100 # U=R*I -> I=U/R
print("Calculated Emitter current: %f A" % Ie)

Ib = (10 - 0.6) / 100000
print("Calculated Base current: %f A" % Ib)
print("Calculated current: %f A" % (Ib+Ie))



In [ ]:
circuit = Circuit('DC led driver')
circuit.include(spice_library['BC547B'])
circuit.include(spice_library['LED2'])

circuit.V('1', '+5V', circuit.gnd, 'DC 5')
circuit.V('2', 'IN', circuit.gnd, 'DC 10 AC 5V SIN(0 5V 1k)')

circuit.BJT('1', 'col', 'base', 'em', model="BC547B")

circuit.R(1, 'IN', 'base', 100@u_kΩ)
R = circuit.R(2, '+5V', 'col', 5.1@u_kΩ)

circuit.D(1, 'em', circuit.gnd, model="LED2")

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.options('savecurrents')

op = simulator.operating_point()
print("Emitter Current: %f" % op['@Q1[ie]'])

analysis  = simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=110@u_ms)

#plot the figure
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time (ms)')
ax1_buffer.set_ylabel('IN [V]')
#ax1_buffer.plot(u_ms(analysis['IN'].abscissa), analysis['IN'], color='Blue')
#ax1_buffer.plot(u_ms(analysis['em'].abscissa), analysis['em'], color='Red')
ax1_buffer.plot(u_ms(analysis['@Q1[ie]'].abscissa), analysis['@Q1[ie]'], color='Green')
ax1_buffer.legend(('Vin [V]', 'Vout (R2=100 kΩ)', 'Vout (R2=125 kΩ)'), loc=(0.01,0.65))

plt.tight_layout()
plt.show()

# ac led driver


In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=0.9)

d.add(elm.Dot('left', lftlabel='IN', open=True))
d.add(elm.Resistor(label='$R_1$\n12k'))
D = d.add(elm.Diode(label='$D_1$'))
d.add(elm.Dot())
d.add(elm.Line(l=d.unit/4))
Q = d.add(elm.BjtNpn(circle='True', label='$Q_1$', to='base'))

d.add(elm.Capacitor("down", label="$C_1$\n100n", at=D.end))
d.add(elm.Ground())

d.add(elm.LED("down", label='$D_1$', at=Q.emitter))
d.add(elm.Resistor(label='$R_2$\n470'))
d.add(elm.Ground)

L = d.add(elm.Line("up", l=d.unit/4, at=Q.collector))
d.add(elm.Dot())
d.add(elm.Line("up", l=d.unit/4))
d.add(elm.Vdd(label="+15V"))
d.add(elm.Capacitor("right", at=L.end, label="$C_2$\n100n"))
d.add(elm.Ground())

d.draw()

In [ ]:
circuit = Circuit('ac led driver')
circuit.include(spice_library['BC547B'])
circuit.include(spice_library['D1N4148'])
circuit.include(spice_library['LED1'])

circuit.V('1', '+5V', circuit.gnd, 'DC 5')
circuit.V('2', 'IN', circuit.gnd, 'DC 5 AC 5V SIN(0 5V 1k)')

circuit.BJT('1', '+5V', 'base', 'em', model="BC547B")

circuit.R(1, 'IN', 'R1', 100@u_kΩ)
circuit.D(1, 'R1', 'base', model="D1N4148")
circuit.C(1, 'base', circuit.gnd, 100@u_nF)

circuit.D(2, 'em', 'D2', model="LED1")
R = circuit.R(2, 'D2', circuit.gnd, 470@u_Ω)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis  = simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=110@u_ms)

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time (ms)')
ax1_buffer.set_ylabel('IN [V]')
ax1_buffer.plot(u_ms(analysis['IN'].abscissa), analysis['IN'], color='Blue')
ax1_buffer.plot(u_ms(analysis['em'].abscissa), analysis['em'])
ax1_buffer.legend(('Vin [V]', 'Vout (R2=100 kΩ)', 'Vout (R2=125 kΩ)'), loc=(0.01,0.65))

plt.tight_layout()
plt.show()

# bipolar led driver

In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=0.9)

O = d.add(elm.Opamp())
d.add(elm.Line('left', l=d.unit, xy=O.in2))
d.add(elm.Dot(lftlabel='IN', open=True))

d.add(elm.Line('left', l=d.unit/4, xy=O.in1))
d.add(elm.Line('up', l=d.unit/2))
d.add(elm.Line('right', tox=O.out+d.unit/4))
d.add(elm.Line('down', toy=O.out))
D = d.add(elm.Dot())
d.add(elm.Line('left', tox=O.out))

d.add(elm.Line('right', l=d.unit/4, xy=D.end))
d.add(elm.Resistor(label='$R_1$\n12k'))
d.add(elm.LED(label='$D_1$'))
d.add(elm.Line(l=d.unit/4))
d.add(elm.Ground())

d.draw()

In [ ]:
circuit = Circuit('bipolar led driver')
circuit.include(spice_library['TL072'])
circuit.include(spice_library['LED1'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'IN', circuit.gnd, 'DC 5 AC 5V SIN(0 5V 1k)')

circuit.X('1', 'TL072', 'IN', 'Xo', '+15V', '-15V', 'Xo')
circuit.R(1, 'Xo', 'R1', 1@u_kΩ)
circuit.D(1, 'R1', circuit.gnd, model="LED1")
circuit.D(2, circuit.gnd, 'R1', model="LED1")

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=10@u_ms)

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time (ms)')
ax1_buffer.set_ylabel('IN [V]')
ax1_buffer.plot(u_ms(analysis['IN'].abscissa), analysis['IN'], color='Blue')
ax1_buffer.plot(u_ms(analysis['R1'].abscissa), analysis['R1'], color='Red')
ax1_buffer.legend(('Vin [V]', 'Vout (R2=100 kΩ)', 'Vout (R2=125 kΩ)'), loc=(0.01,0.65))

plt.tight_layout()
plt.show()

The blue line is the input signal (5V p2p) and the red the output signal with 100kOhm resistors. The output is a little less then excpected. For the other signals the resistor R2 is replaced with 125kΩ, 150kΩ, 175kΩ and 200kΩ. With 150kΩ the gain is roughly one. We also see that the ouput is not symetrical. The clipping occures earlier for the negative part of the signal. With higher gain the signal is soft clipped. 

in the simulation R2 is unchanched. For R4 the simulation is done with 100kΩ, 500kΩ, 1MΩ and 10MΩ.

# references

1. [CD4069](https://www.ti.com/lit/ds/schs054e/schs054e.pdf) Datasheet
2. [UBE Screamer](http://www.runoffgroove.com/ubescreamer.html) TS-808 workalike built around the CD4049UBE logic IC
3. [WASP Filter](https://www.schmitzbits.de/wasp.html) Jürgen Haibles clone of the EDP Wasp VCF from René Schmitz.